<a href="https://colab.research.google.com/github/shivani-jadhao/GithubTest/blob/main/Crowd_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
!pip install ultralytics

In [ ]:
from ultralytics import YOLO

# Training YOLOv8 model to detect only people
model = YOLO("yolov8x.pt",task = "detect")
model.train(data = "coco128.yaml",epochs = 10,device = 0,pretrained = True,seed = 666,single_cls = True,augment = True,classes = [0])

In [ ]:
import cv2, math, numpy as np

def box_dist(box1,box2):
  """
  Summary: This function calculates the Euclidean distance between the centers of two bounding boxes.
  Arguments:
    box1 (tuple): A tuple representing the coordinates of the first bounding box in the format (x1, y1, x2, y2), 
    where (x1, y1) is the top-left corner and (x2, y2) is the bottom-right corner of the box.
    box2 (tuple): A tuple representing the coordinates of the second bounding box in the same format as box1.
  Returns:
    distance (float): A float representing the Euclidean distance between the centers of box1 and box2.
  Explanation: The function takes in two bounding boxes and calculates the center coordinates of each box. 
  It then calculates the Euclidean distance between the centers of the two boxes and returns the distance as a float.

  """
  x1, y1, x2, y2 = box1
  x3, y3, x4, y4 = box2
  cx1, cy1 = x1 + x2 / 2, y1 + y2 / 2
  cx2, cy2 = x3 + x4 / 2, y3 + y4 / 2
  dx, dy = cx1 - cx2, cy1 - cy2
  
  return np.sqrt(dx ** 2 + dy ** 2)

def overlap(box1,box2):
  """
  Summary: This function checks if two bounding boxes overlap.
  Arguments:
    box1 (tuple): A tuple representing the coordinates of the first bounding box in the format (x1, y1, x2, y2), 
    where (x1, y1) is the top-left corner and (x2, y2) is the bottom-right corner of the box.
    box2 (tuple): A tuple representing the coordinates of the second bounding box in the same format as box1.
  Returns:
    overlap (bool): A boolean value indicating whether the two boxes overlap.
  Explanation: The function takes in two bounding boxes and checks if they overlap by comparing the x and y coordinates of the boxes. 
  If any of the sides of one box lies outside the other box, then they do not overlap and the function returns False. Otherwise, the function returns True.

  """
  x1, y1, x2, y2 = box1
  a1, b1, a2, b2 = box2
  if x1 > a2 or a1 > x2:
    return False
  if y1 > b2 or b1 > y2:
    return False
  
  return True

def remove_overlapping_boxes(boxes):
  """
  Summary: This function removes overlapping bounding boxes from a list of boxes.
  Arguments:
    boxes (list): A list of bounding boxes, where each bounding box is represented as a tuple in the format (x1, y1, x2, y2), 
    where (x1, y1) is the top-left corner and (x2, y2) is the bottom-right corner of the box.
  Returns:
    result_boxes (list): A list of non-overlapping bounding boxes, where each bounding box is represented as a tuple in the same format as the input boxes.
  Explanation: The function takes in a list of bounding boxes and sorts them by their area in descending order. 
  It then iterates through the sorted boxes, ignoring any boxes that have already been marked for removal and checks if they overlap with any of 
  the remaining boxes. If they do overlap, it removes the box with the smaller area, and if they do not overlap, 
  it adds the box to the list of non-overlapping boxes. The function returns the list of non-overlapping boxes.

  """
  sorted_boxes = sorted(boxes,key = lambda box: (box[2] - box[0]) * (box[3] - box[1]),reverse = True)
  result_boxes = []
  ignore = set()
  for i in range(len(sorted_boxes)):
    if i in ignore:
      continue
    for j in range(i+1,len(sorted_boxes)):
      if j in ignore:
        continue
      if overlap(sorted_boxes[i],sorted_boxes[j]):
        if (sorted_boxes[i][2] - sorted_boxes[i][0]) * (sorted_boxes[i][3] - sorted_boxes[i][1])\
         >= (sorted_boxes[j][2] - sorted_boxes[j][0]) * (sorted_boxes[j][3] - sorted_boxes[j][1]):
          ignore.add(j)
        else:
          ignore.add(i)
          break
    else:
      result_boxes.append(sorted_boxes[i])
  
  return result_boxes

In [ ]:
# Initializing a video capture object to read from the input video file
cap = cv2.VideoCapture("/content/drive/MyDrive/PTV_3.mp4")

# Creating a VideoWriter object to write to the output video file
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter("output_video.mp4",fourcc,30.0,(int(cap.get(3)),int(cap.get(4))))

# Looping through each frame in the video
while True:
    ret, frm = cap.read()
    if not ret:
        break

    # Converting the frame to RGB format
    img_rgb = cv2.cvtColor(frm,cv2.COLOR_BGR2RGB)

    # Predicting object detection results for the current frame using the trained model
    res = model.predict(img_rgb,conf = 0.3,device = 0,line_thickness = 5,classes = 0)
    pred = res[0].cpu()
    pred_bbox = pred.boxes.xyxy

    # Removing overlapping bounding boxes
    pred_bbox = remove_overlapping_boxes(pred_bbox)

    # Detecting crowded bounding boxes
    crowd_boxes = set()
    for i in range(len(pred_bbox)):
        for j in range(i+1,len(pred_bbox)):
            dist = box_dist(pred_bbox[i],pred_bbox[j])
            if dist < 100:
                crowd_boxes.add(i)
                crowd_boxes.add(j)

    # Drawing bounding boxes on the frame
    for i,bbox in enumerate(pred_bbox):
        x1, y1, x2, y2 = map(int,bbox)
        if i in crowd_boxes and len(crowd_boxes) >= 3:
            cv2.rectangle(frm,(x1,y1),(x2,y2),(0,0,255),5)
        else:
            cv2.rectangle(frm,(x1,y1),(x2,y2),(0,255,0),5)

    # Adding a text message to the frame if a crowd is detected
    crowd_count = len(crowd_boxes)
    if crowd_count >= 3:
        text = f"CROWD DETECTED! {crowd_count} people."
        cv2.putText(img = frm,text = text,org = (50,50),fontFace = cv2.FONT_HERSHEY_SIMPLEX,fontScale = 1,
                    color = (0,0,255),thickness = 5,lineType = cv2.LINE_AA)

    # Writing the modified frame to the output video file
    out.write(frm)

# Releasing the video capture and VideoWriter objects
cap.release()
out.release()
